In [1]:
import os
from tqdm import tqdm
import numpy as np

import torch
import torch.optim as optim

from utils import threshold, criterion, unnorm
from dataset_utils import imagenet_loader
from models import load_model
from torchvision.utils import save_image

# Configure Script
gpu_num = 1
epochs = [100, 200, 300, 500, 1000]
batch_size = 4
eps = 16 / 255
lr = 0.02
n_images = 100
model_flag = 'imagebind'
gamma_epochs = 100
modality = 'thermal'

if type(epochs) == list:
    max_epochs = max(epochs)
else:
    max_epochs = epochs
    epochs = [epochs]

device = f"cuda:{gpu_num}" if torch.cuda.is_available() and gpu_num >= 0 else "cpu"
assert n_images % batch_size == 0

# Instantiate Model
model = load_model(model_flag, device)

/home/rishi/anaconda3/envs/adv_collisions/lib/python3.11/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/home/rishi/anaconda3/envs/adv_collisions/lib/python3.11/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/rishi/anaconda3/envs/adv_collisions/lib/python3.11/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.t

In [2]:
from torchvision import transforms
from PIL import Image

def load_and_transform_thermal_data(image_paths, device):
    if image_paths is None:
        return None

    image_outputs = []
    for image_path in image_paths:
        data_transform = transforms.Compose(
            [
                transforms.Resize(
                    224, interpolation=transforms.InterpolationMode.BICUBIC
                ),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=(0.48145466, 0.4578275, 0.40821073),
                    std=(0.26862954, 0.26130258, 0.27577711),
                ),
                transforms.Grayscale()
            ]
        )
        with open(image_path, "rb") as fopen:
            image = Image.open(fopen).convert("RGB")

        image = data_transform(image).to(device)
        image_outputs.append(image)
    return torch.stack(image_outputs, dim=0)

In [3]:
# Define the paths for the original and perturbed folders
original_folder = '../data/thermal'
perturbed_folder = 'outputs/thermal_perturbed'

# Get the list of image files in the original folder
image_files = [file for file in os.listdir(original_folder) if file.endswith('.jpg')]

# Iterate over each image file
for image_file in image_files:
    # Load the original image
    perturbed_image_path = os.path.join(original_folder, image_file)
    X = load_and_transform_thermal_data([perturbed_image_path], device)

    X_init = X.clone().detach().cpu().requires_grad_(False)
    target_text=["murderer with gun",]
    Y = model.forward(target_text, "text", normalize=False)
    X, Y = X.to(device).requires_grad_(True), Y.to(device)

    max_epochs=2000
    pbar = tqdm(range(max_epochs))
    X_init = X.clone().detach().cpu().requires_grad_(False)
    X, Y = X.to(device).requires_grad_(True), Y.to(device)
    X_max, X_min = threshold(X, eps, modality, device)
    optimizer = optim.SGD([X], lr=lr)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer,
                                                np.arange(gamma_epochs, max_epochs, gamma_epochs),
                                                gamma=0.9)
    for j in pbar:
        eta = scheduler.get_last_lr()[0]
        embeds = model.forward(X, modality, normalize=False)
        cton = 1 - criterion(embeds, Y, dim=1).detach().cpu()
        loss = 1 - criterion(embeds, Y, dim=1)
        update = eta * torch.autograd.grad(outputs=loss.mean(), inputs=X)[0].sign()
        X = (X.detach().cpu() - update.detach().cpu()).to(device)
        X = torch.clamp(X, min=X_min, max=X_max).requires_grad_(True)
        pbar.set_postfix({'loss': cton, 'eta': eta})
        scheduler.step()

    perturbed_image_path = os.path.join(perturbed_folder, image_file.replace('.jpg','.png'))
    save_image(unnorm(torch.squeeze(X.cuda()))[0], perturbed_image_path)

print("Perturbation completed and images saved to the perturbed folder.")


  0%|          | 0/2000 [00:00<?, ?it/s]

torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 224, 224])


  0%|          | 0/2000 [00:04<?, ?it/s, loss=tensor([0.9595]), eta=0.02]/home/rishi/anaconda3/envs/adv_collisions/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
 20%|█▉        | 396/2000 [00:13<00:56, 28.32it/s, loss=tensor([0.3746]), eta=0.0146]


KeyboardInterrupt: 